## Byte-Pair Encoding (BPE)

Subword-based tokenization is a bridge between word and character-based tokenization. The main idea is to solve the issues faced by word-based tokenization (very large vocabulary size, large number of OOV tokens, and different meaning of very similar words) and character-based tokenization (very long sequences and less meaningful individual tokens).

**Byte-pair encoding** is one of the popular among the sub-word based tokenization. It was first introduced in 1994 as a simple data compression technique by iteratively replacing the most frequent pair of bytes in a sequence with a single, unused byte. It has been adapted as tokenization algorithm and used in most of language models like BERT and GPT. 

> In this notebook, we explore the minimalist working of BPE. Here, instead of bytes, BPE merges frequent characters or character sequences.

In [1]:
import collections

In [2]:
Text = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
    f"There is an 80% chance of rainfall today. We are pretty sure it is going to rain."] # Text corpus
word_freq = collections.defaultdict(int) # Computes count of each individual words.
for text in Text:
    new_words = text.split()
    for word in new_words:
        word = " ".join(word.strip()) + " </w>" # Creates space between characters of word. </w> indicates the end of word.
        word_freq[word] += 1

In [3]:
print(word_freq)

defaultdict(<class 'int'>, {'T h i s </w>': 3, 'i s </w>': 4, 't h e </w>': 1, 'H u g g i n g </w>': 1, 'F a c e </w>': 1, 'C o u r s e . </w>': 1, 'c h a p t e r </w>': 1, 'a b o u t </w>': 1, 't o k e n i z a t i o n . </w>': 1, 's e c t i o n </w>': 1, 's h o w s </w>': 1, 's e v e r a l </w>': 1, 't o k e n i z e r </w>': 1, 'a l g o r i t h m s . </w>': 1, 'H o p e f u l l y , </w>': 1, 'y o u </w>': 1, 'w i l l </w>': 1, 'b e </w>': 1, 'a b l e </w>': 1, 't o </w>': 2, 'u n d e r s t a n d </w>': 1, 'h o w </w>': 1, 't h e y </w>': 1, 'a r e </w>': 2, 't r a i n e d </w>': 1, 'a n d </w>': 1, 'g e n e r a t e </w>': 1, 't o k e n s . </w>': 1, 'T h e r e </w>': 1, 'a n </w>': 1, '8 0 % </w>': 1, 'c h a n c e </w>': 1, 'o f </w>': 1, 'r a i n f a l l </w>': 1, 't o d a y . </w>': 1, 'W e </w>': 1, 'p r e t t y </w>': 1, 's u r e </w>': 1, 'i t </w>': 1, 'g o i n g </w>': 1, 'r a i n . </w>': 1})


In [4]:
for word, freq in word_freq.items():
    print(word.split())
    break

['T', 'h', 'i', 's', '</w>']


In [5]:
"""Creates a vocabulary of individual characters along with count"""
vocab_dic = collections.defaultdict(int)

for word, freq in word_freq.items():
    for letter in word.split():
            vocab_dic[letter] += freq

print(vocab_dic)
print(len(vocab_dic))

defaultdict(<class 'int'>, {'T': 4, 'h': 11, 'i': 19, 's': 16, '</w>': 48, 't': 19, 'e': 28, 'H': 2, 'u': 7, 'g': 7, 'n': 16, 'F': 1, 'a': 19, 'c': 5, 'C': 1, 'o': 17, 'r': 15, '.': 6, 'p': 3, 'b': 3, 'k': 3, 'z': 2, 'w': 3, 'v': 1, 'l': 9, 'm': 1, 'f': 3, 'y': 5, ',': 1, 'd': 5, '8': 1, '0': 1, '%': 1, 'W': 1})
34


In [6]:
"""Create the split of each words of the text corpus"""
splits = {word: word.split() for word in word_freq.keys()}
print(splits)

{'T h i s </w>': ['T', 'h', 'i', 's', '</w>'], 'i s </w>': ['i', 's', '</w>'], 't h e </w>': ['t', 'h', 'e', '</w>'], 'H u g g i n g </w>': ['H', 'u', 'g', 'g', 'i', 'n', 'g', '</w>'], 'F a c e </w>': ['F', 'a', 'c', 'e', '</w>'], 'C o u r s e . </w>': ['C', 'o', 'u', 'r', 's', 'e', '.', '</w>'], 'c h a p t e r </w>': ['c', 'h', 'a', 'p', 't', 'e', 'r', '</w>'], 'a b o u t </w>': ['a', 'b', 'o', 'u', 't', '</w>'], 't o k e n i z a t i o n . </w>': ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', '.', '</w>'], 's e c t i o n </w>': ['s', 'e', 'c', 't', 'i', 'o', 'n', '</w>'], 's h o w s </w>': ['s', 'h', 'o', 'w', 's', '</w>'], 's e v e r a l </w>': ['s', 'e', 'v', 'e', 'r', 'a', 'l', '</w>'], 't o k e n i z e r </w>': ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r', '</w>'], 'a l g o r i t h m s . </w>': ['a', 'l', 'g', 'o', 'r', 'i', 't', 'h', 'm', 's', '.', '</w>'], 'H o p e f u l l y , </w>': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y', ',', '</w>'], 'y o u </w>': ['y', 'o

In [7]:
"""Function to compute the character pair along with counts"""
def compute_pair_freqs(splits):
    pair_freqs = collections.defaultdict(int)
    for word, freq in word_freq.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [8]:
pair_freqs = compute_pair_freqs(splits) # Computes the pair counts from the splits.
print(pair_freqs)

defaultdict(<class 'int'>, {('T', 'h'): 4, ('h', 'i'): 3, ('i', 's'): 7, ('s', '</w>'): 8, ('t', 'h'): 3, ('h', 'e'): 3, ('e', '</w>'): 11, ('H', 'u'): 1, ('u', 'g'): 1, ('g', 'g'): 1, ('g', 'i'): 1, ('i', 'n'): 5, ('n', 'g'): 2, ('g', '</w>'): 2, ('F', 'a'): 1, ('a', 'c'): 1, ('c', 'e'): 2, ('C', 'o'): 1, ('o', 'u'): 3, ('u', 'r'): 2, ('r', 's'): 2, ('s', 'e'): 3, ('e', '.'): 1, ('.', '</w>'): 6, ('c', 'h'): 2, ('h', 'a'): 2, ('a', 'p'): 1, ('p', 't'): 1, ('t', 'e'): 2, ('e', 'r'): 6, ('r', '</w>'): 2, ('a', 'b'): 2, ('b', 'o'): 1, ('u', 't'): 1, ('t', '</w>'): 2, ('t', 'o'): 6, ('o', 'k'): 3, ('k', 'e'): 3, ('e', 'n'): 4, ('n', 'i'): 2, ('i', 'z'): 2, ('z', 'a'): 1, ('a', 't'): 2, ('t', 'i'): 2, ('i', 'o'): 2, ('o', 'n'): 2, ('n', '.'): 2, ('e', 'c'): 1, ('c', 't'): 1, ('n', '</w>'): 2, ('s', 'h'): 1, ('h', 'o'): 2, ('o', 'w'): 2, ('w', 's'): 1, ('e', 'v'): 1, ('v', 'e'): 1, ('r', 'a'): 5, ('a', 'l'): 3, ('l', '</w>'): 3, ('z', 'e'): 1, ('l', 'g'): 1, ('g', 'o'): 2, ('o', 'r'): 1, ('

In [9]:
# Gives the best pair of highest frequency.
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('e', '</w>') 11


In [10]:
"""Function to merge the best pairs"""
def merge_pair(a, b, splits):
    for word in word_freq:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [11]:
"""
BPE algorithm 

1. Create the split of words.
2. Compute frequency of character pairs.
3. Find the most frequently occuring pair.
4. Merge the pair to create a new token.
5. Recalculate the frequency and update the vocabulary.
6. Continue the loop for the given iteration or until it reaches maximum vocabulary size.

"""
splits = {word: word.split() for word in word_freq.keys()} # 1
pair_freqs = compute_pair_freqs(splits) 
vocab_size = 200 # Maximum size of vocabulary
merges = {}
i = 0
while i < 50: # For 50 iterations
    pair_freqs = compute_pair_freqs(splits) # 2
    if not pair_freqs:
        break
    best_pair = max(pair_freqs, key=pair_freqs.get) # 3
    merges[best_pair] = best_pair[0] + best_pair[1] # 4
    # 5
    max_freq = pair_freqs[best_pair]
    vocab_dic[best_pair[0] + best_pair[1]] = max_freq
    vocab_dic[best_pair[0]] -= max_freq
    vocab_dic[best_pair[1]] -= max_freq
    if vocab_dic[best_pair[0]] == 0: 
        vocab_dic.pop(best_pair[0])
    if vocab_dic[best_pair[1]] == 0:
        vocab_dic.pop(best_pair[1])
    splits = merge_pair(*best_pair, splits)
    if i % 10 == 0: # Length of vocabulary for each 10 iterations
        print('Vocab len ', len(vocab_dic))
    if len(vocab_dic) == vocab_size:
        break
    
    i += 1

Vocab len  35
Vocab len  43
Vocab len  50
Vocab len  56
Vocab len  63


In [12]:
len(vocab_dic)

66

In [13]:
print(merges)

{('e', '</w>'): 'e</w>', ('s', '</w>'): 's</w>', ('i', 's</w>'): 'is</w>', ('.', '</w>'): '.</w>', ('e', 'r'): 'er', ('t', 'o'): 'to', ('i', 'n'): 'in', ('T', 'h'): 'Th', ('e', 'n'): 'en', ('a', 'n'): 'an', ('Th', 'is</w>'): 'This</w>', ('t', 'h'): 'th', ('o', 'u'): 'ou', ('s', 'e'): 'se', ('to', 'k'): 'tok', ('tok', 'en'): 'token', ('a', 'l'): 'al', ('d', '</w>'): 'd</w>', ('r', 'e</w>'): 're</w>', ('r', 'a'): 'ra', ('ra', 'in'): 'rain', ('in', 'g'): 'ing', ('ing', '</w>'): 'ing</w>', ('c', 'e</w>'): 'ce</w>', ('c', 'h'): 'ch', ('er', '</w>'): 'er</w>', ('a', 'b'): 'ab', ('t', '</w>'): 't</w>', ('token', 'i'): 'tokeni', ('tokeni', 'z'): 'tokeniz', ('a', 't'): 'at', ('i', 'o'): 'io', ('io', 'n'): 'ion', ('h', 'o'): 'ho', ('ho', 'w'): 'how', ('g', 'o'): 'go', ('s', '.</w>'): 's.</w>', ('l', 'l'): 'll', ('to', '</w>'): 'to</w>', ('an', 'd</w>'): 'and</w>', ('y', '</w>'): 'y</w>', ('a', 're</w>'): 'are</w>', ('th', 'e</w>'): 'the</w>', ('H', 'u'): 'Hu', ('Hu', 'g'): 'Hug', ('Hug', 'g'): '

In [14]:
print(vocab_dic)

defaultdict(<class 'int'>, {'i': 3, 's': 3, '</w>': 10, 't': 6, 'e': 4, 'H': 1, 'u': 3, 'g': 1, 'n': 1, 'a': 2, 'c': 1, 'o': 2, 'r': 3, 'p': 3, 'b': 1, 'w': 1, 'v': 1, 'l': 2, 'm': 1, 'f': 3, 'y': 3, ',': 1, 'd': 2, '8': 1, '0': 1, '%': 1, 'W': 1, 'e</w>': 5, 's</w>': 1, 'is</w>': 4, '.</w>': 4, 'er': 4, 'to': 1, 'Th': 1, 'en': 1, 'an': 2, 'This</w>': 3, 'th': 2, 'ou': 2, 'se': 3, 'token': 1, 'al': 3, 'd</w>': 1, 're</w>': 1, 'rain': 3, 'ing</w>': 1, 'ce</w>': 1, 'ch': 2, 'er</w>': 2, 'ab': 2, 't</w>': 2, 'tokeniz': 2, 'at': 2, 'ion': 2, 'how': 2, 'go': 2, 's.</w>': 2, 'll': 2, 'to</w>': 2, 'and</w>': 2, 'y</w>': 2, 'are</w>': 2, 'the</w>': 1, 'Hugging</w>': 1, 'Face</w>': 1, 'Cou': 1})


In [15]:
vocab = list(vocab_dic.keys()) # Creating final vocabulary with <unk> token.
vocab.append("<unk>")
len(vocab)

67

In [16]:
vocab.sort()
vocab_dict = {key: i for i, key in enumerate(vocab)} # Creating the vocabulary dictionary to create numerical tokens.
print(vocab_dict)

{'%': 0, ',': 1, '.</w>': 2, '0': 3, '8': 4, '</w>': 5, '<unk>': 6, 'Cou': 7, 'Face</w>': 8, 'H': 9, 'Hugging</w>': 10, 'Th': 11, 'This</w>': 12, 'W': 13, 'a': 14, 'ab': 15, 'al': 16, 'an': 17, 'and</w>': 18, 'are</w>': 19, 'at': 20, 'b': 21, 'c': 22, 'ce</w>': 23, 'ch': 24, 'd': 25, 'd</w>': 26, 'e': 27, 'e</w>': 28, 'en': 29, 'er': 30, 'er</w>': 31, 'f': 32, 'g': 33, 'go': 34, 'how': 35, 'i': 36, 'ing</w>': 37, 'ion': 38, 'is</w>': 39, 'l': 40, 'll': 41, 'm': 42, 'n': 43, 'o': 44, 'ou': 45, 'p': 46, 'r': 47, 'rain': 48, 're</w>': 49, 's': 50, 's.</w>': 51, 's</w>': 52, 'se': 53, 't': 54, 't</w>': 55, 'th': 56, 'the</w>': 57, 'to': 58, 'to</w>': 59, 'token': 60, 'tokeniz': 61, 'u': 62, 'v': 63, 'w': 64, 'y': 65, 'y</w>': 66}


In [17]:
"""Function to tokenize the given input text"""
def tokenize(text):
    tokens2int = lambda x: vocab_dict.get(x) if x in vocab_dict.keys() else vocab_dict.get('<unk>')
    splits = text.split(" ")
    splits = [" ".join(word.strip()) + " </w>" for word in splits]
    splits = [word.split() for word in splits]
    print("Initial split of the input text: ", splits)
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split
    print("Split after applying tokenization: ", splits)
    splits = [[tokens2int(c) for c in split] for split in splits]

    return sum(splits, [])

In [18]:
"""Function to de-tokenize the numerical tokens"""
def detokenize(tokens):
    int2tokens = lambda x: list(vocab_dict.keys())[list(vocab_dict.values()).index(x)]
    text = [int2tokens(c) for c in tokens]
    return text

In [19]:
Text = ["This is to test the tokenizer, how it tokenizes the given text corpus.",
        "Learn the basics to lay strong foundation."]

In [20]:
Tokens = []
for text in Text:
    tokens = tokenize(text)
    Tokens.append(tokens)
    print()

Initial split of the input text:  [['T', 'h', 'i', 's', '</w>'], ['i', 's', '</w>'], ['t', 'o', '</w>'], ['t', 'e', 's', 't', '</w>'], ['t', 'h', 'e', '</w>'], ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r', ',', '</w>'], ['h', 'o', 'w', '</w>'], ['i', 't', '</w>'], ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 's', '</w>'], ['t', 'h', 'e', '</w>'], ['g', 'i', 'v', 'e', 'n', '</w>'], ['t', 'e', 'x', 't', '</w>'], ['c', 'o', 'r', 'p', 'u', 's', '.', '</w>']]
Split after applying tokenization:  [['This</w>'], ['is</w>'], ['to</w>'], ['t', 'e', 's', 't</w>'], ['the</w>'], ['tokeniz', 'er', ',', '</w>'], ['how', '</w>'], ['i', 't</w>'], ['tokeniz', 'e', 's</w>'], ['the</w>'], ['g', 'i', 'v', 'en', '</w>'], ['t', 'e', 'x', 't</w>'], ['c', 'o', 'r', 'p', 'u', 's.</w>']]

Initial split of the input text:  [['L', 'e', 'a', 'r', 'n', '</w>'], ['t', 'h', 'e', '</w>'], ['b', 'a', 's', 'i', 'c', 's', '</w>'], ['t', 'o', '</w>'], ['l', 'a', 'y', '</w>'], ['s', 't', 'r', 'o', 'n', 'g', '</w>'], ['f', 'o', 

In [21]:
print(Tokens)

[[12, 39, 59, 54, 27, 50, 55, 57, 61, 30, 1, 5, 35, 5, 36, 55, 61, 27, 52, 57, 33, 36, 63, 29, 5, 54, 27, 6, 55, 22, 44, 47, 46, 62, 51], [6, 27, 14, 47, 43, 5, 57, 21, 14, 50, 36, 22, 52, 59, 40, 14, 66, 50, 54, 47, 44, 43, 33, 5, 32, 45, 43, 25, 20, 38, 2]]


In [22]:
for token in Tokens:
    out = detokenize(token)
    print("After de-tokenization: ", out)
    out = "".join(out)
    print("Intermediate text: ", out)
    out = "".join(out).replace("</w>", " ").strip()
    print("Final text after tokenization: ", out) 
    print()

After de-tokenization:  ['This</w>', 'is</w>', 'to</w>', 't', 'e', 's', 't</w>', 'the</w>', 'tokeniz', 'er', ',', '</w>', 'how', '</w>', 'i', 't</w>', 'tokeniz', 'e', 's</w>', 'the</w>', 'g', 'i', 'v', 'en', '</w>', 't', 'e', '<unk>', 't</w>', 'c', 'o', 'r', 'p', 'u', 's.</w>']
Intermediate text:  This</w>is</w>to</w>test</w>the</w>tokenizer,</w>how</w>it</w>tokenizes</w>the</w>given</w>te<unk>t</w>corpus.</w>
Final text after tokenization:  This is to test the tokenizer, how it tokenizes the given te<unk>t corpus.

After de-tokenization:  ['<unk>', 'e', 'a', 'r', 'n', '</w>', 'the</w>', 'b', 'a', 's', 'i', 'c', 's</w>', 'to</w>', 'l', 'a', 'y</w>', 's', 't', 'r', 'o', 'n', 'g', '</w>', 'f', 'ou', 'n', 'd', 'at', 'ion', '.</w>']
Intermediate text:  <unk>earn</w>the</w>basics</w>to</w>lay</w>strong</w>foundation.</w>
Final text after tokenization:  <unk>earn the basics to lay strong foundation.



### References

1. [Hugging Face](https://huggingface.co/learn/nlp-course/en/chapter6/5).
2. [Blog](https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0), [Medium](https://medium.com/@hsinhungw/understanding-byte-pair-encoding-fd196ebfe93f).
3. [Freecodecamp](https://www.freecodecamp.org/news/evolution-of-tokenization/).